In [1]:
# Libraries
import numpy as np
import pandas as pd

In [35]:
# Data import
gem1 = pd.read_csv('../data/gem_1.csv')
gem2 = pd.read_csv('../data/gem_2.csv')


# Concatenation
combined_df = pd.concat([gem1, gem2], ignore_index=True)
combined_df

,Line,Sample,Easting [m],Northing [m],Mark,Status,GPSStat,GPSalt,Time[ms],Time[hhmmss.sss],...,Q_450Hz [ppm],I_1530Hz [ppm],Q_1530Hz [ppm],I_5310Hz [ppm],Q_5310Hz [ppm],I_18330Hz [ppm],Q_18330Hz [ppm],I_63030Hz [ppm],Q_63030Hz [ppm],Qsum [ppm]
0,1,5,344123.35,4482801.47,0,0,4,2114.74,66603914.2,183003.9142,...,771.0,778.0,797.0,1230.0,1490.0,1770.0,4060.0,4410.0,11800.0,18900.0
1,1,6,344123.35,4482801.47,0,0,4,2114.74,66604014.2,183004.0142,...,671.0,746.0,786.0,1170.0,1470.0,1790.0,4090.0,4320.0,11800.0,18800.0
2,1,7,344123.35,4482801.48,0,0,4,2114.74,66604114.2,183004.1142,...,665.0,679.0,835.0,1180.0,1560.0,1840.0,4190.0,4300.0,11700.0,19000.0
3,1,8,344123.35,4482801.48,0,0,4,2114.74,66604214.2,183004.2142,...,722.0,691.0,832.0,1250.0,1620.0,1890.0,4190.0,4410.0,11700.0,19100.0
4,1,9,344123.35,4482801.48,0,0,4,2114.74,66604314.2,183004.3142,...,830.0,715.0,806.0,1210.0,1640.0,1870.0,4110.0,4360.0,11700.0,19100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57756,0,44062,344146.16,4482830.60,0,0,4,2099.40,73069936.1,201749.9361,...,667.0,365.0,772.0,653.0,1170.0,935.0,3230.0,2720.0,9600.0,15400.0
57757,0,44063,344146.16,4482830.60,0,0,4,2099.40,73070036.1,201750.0361,...,641.0,333.0,800.0,618.0,1220.0,1010.0,3240.0,2830.0,9660.0,15600.0
57758,0,44064,344146.16,4482830.60,0,0,4,2099.40,73070136.1,201750.1361,...,650.0,363.0,761.0,653.0,1190.0,1040.0,3250.0,2890.0,9720.0,15600.0
57759,0,44065,344146.15,4482830.60,0,0,4,2099.40,73070236.1,201750.2361,...,786.0,366.0,719.0,675.0,1190.0,1030.0,3220.0,2890.0,9690.0,15600.0


In [47]:
## Rename dataframe columns to comply with EMagPy's requirements
# Drops unnecessary columns
dropped =  combined_df.drop(['Line', 'Sample', 'Mark', 'Status', 'GPSStat', 'PowerLn [mG]', 'Time[ms]', 'Time[hhmmss.sss]', 'I_450Hz [ppm]', 'I_1530Hz [ppm]', 'I_5310Hz [ppm]', 'I_18330Hz [ppm]', 'I_63030Hz [ppm]', 'Qsum [ppm]'], axis = 1)
dropped

# Renames columns
old_cols = dropped.columns.to_numpy(dtype = 'str_')
freqs = [450, 1530, 5310, 18330, 63030]
sep = '1.6' # Transmitter/Reciever offset [m]
h = '0.05' # Assumed constant height above ground [m]
new_cols = np.array(['x','y','elevation'])
for i in range(len(freqs)):
    new_cols = np.append(new_cols, f'HCP{sep}f{freqs[i]}h{h}_quad')
col_dict = {}
for old,new in zip(old_cols, new_cols):
    col_dict[str(old)] = str(new)
renamed = dropped.rename(columns = col_dict)
renamed
renamed.shape

(57761, 8)

In [57]:
# Calculates apparent resistivities
mu = 4*np.pi * (10**(-7))# Assumes mu = mu_0
rho = 1.6 # separation
res = np.zeros((len(freqs), renamed.shape[0]))
for i in range(len(freqs)):
    renamed[new_cols[i+3]] = (renamed[new_cols[i]] * (10**-6) * 4) / (freqs[i] * 2 * np.pi * mu * rho**2) # mS/m

In [58]:
# Write to csv
renamed.to_csv('../data/gem2_for_emagpy.csv')